# current code

In [20]:
import numpy as np
import matplotlib.pyplot as plt

from joblib import Parallel, delayed, parallel_backend
from psiam_tied_dv_map_utils import psiam_tied_data_gen_wrapper_V_A_change
import pandas as pd
from numba import prange 
V_A = 1.1
theta_A = 1.8

rate_lambda = 0.14
T_0 = 0.66 * (1e-3)
theta_E = 40.5

t_A_aff = 0.03
t_E_aff = 0.05
t_motor = 0.03

Z_E = 0
L = 0.5

N_sim = int(100000)
dt = 1e-4

step_size = 0.1
N_print = int(1000)
N_params = 1

og_df = pd.read_csv('../out_LED.csv')
df = og_df[ og_df['repeat_trial'].isin([0,2]) | og_df['repeat_trial'].isna() ]
session_type = 7    
df = df[ df['session_type'].isin([session_type]) ]
training_level = 16
df = df[ df['training_level'].isin([training_level]) ]

t_stim_and_led_tuple = [(row['intended_fix'], row['LED_onset_time']) for _, row in df.iterrows()]

ABL_arr = df['ABL'].unique()
ABL_arr.sort()

ILD_arr = df['ILD'].unique()
ILD_arr.sort()

# percentage of LED on trials
frac_of_led = df['LED_trial'].values.sum() / df['LED_trial'].values.size
print(f'frac_of_led: {frac_of_led}')

V_A_change_results_dict = {}
for i in range(N_params):
    new_V_A = V_A + step_size*i

    with parallel_backend('threading', n_jobs=1):
        sim_results = Parallel()(
            delayed(psiam_tied_data_gen_wrapper_V_A_change)(V_A, theta_A, ABL_arr, ILD_arr, rate_lambda, T_0, theta_E, Z_E, t_A_aff, t_E_aff, t_motor, L,t_stim_and_led_tuple, new_V_A, iter_num, N_print, dt)
            for iter_num in prange(N_sim)
        )

    print(f'V_A: {V_A}, new_V_A: {new_V_A}')

    V_A_change_results_dict[i] = {'results': sim_results, 'new_V_A': new_V_A, 'V_A': V_A, }

frac_of_led: 0.33939811074203746
os id: 50090, In iter_num: 0, ABL: 20, ILD: -8.0, t_stim: 0.242287309527516
os id: 50090, In iter_num: 1000, ABL: 20, ILD: -4.0, t_stim: 0.462437051814096
os id: 50090, In iter_num: 2000, ABL: 60, ILD: 1.0, t_stim: 1.43000210375529
os id: 50090, In iter_num: 3000, ABL: 40, ILD: -2.0, t_stim: 0.203730255454088
os id: 50090, In iter_num: 4000, ABL: 60, ILD: 8.0, t_stim: 0.407674446347132
os id: 50090, In iter_num: 5000, ABL: 40, ILD: -2.0, t_stim: 0.279868562332
os id: 50090, In iter_num: 6000, ABL: 60, ILD: 16.0, t_stim: 0.228980345195716
os id: 50090, In iter_num: 7000, ABL: 60, ILD: -4.0, t_stim: 0.723851498515501
os id: 50090, In iter_num: 8000, ABL: 60, ILD: -2.0, t_stim: 0.25134220950704
os id: 50090, In iter_num: 9000, ABL: 60, ILD: -16.0, t_stim: 0.437068140099315
os id: 50090, In iter_num: 10000, ABL: 20, ILD: 8.0, t_stim: 0.309129238655363
os id: 50090, In iter_num: 11000, ABL: 60, ILD: -2.0, t_stim: 0.247594578030519
os id: 50090, In iter_num: 

# multi processing

In [ ]:
N_print = int(1000)
import multiprocessing

# Function for the simulation
def simulate(iter_num, V_A, theta_A, ABL_arr, ILD_arr, rate_lambda, T_0, theta_E, Z_E, t_A_aff, t_E_aff, t_motor, L, t_stim_and_led_tuple, new_V_A, N_print, dt):
    return psiam_tied_data_gen_wrapper_V_A_change(V_A, theta_A, ABL_arr, ILD_arr, rate_lambda, T_0, theta_E, Z_E, t_A_aff, t_E_aff, t_motor, L, t_stim_and_led_tuple, new_V_A, iter_num, N_print, dt)

# Wrapper function to pass to the pool
def simulate_wrapper(args):
    return simulate(*args)

# Create the results dictionary
V_A_change_results_dict = {}

for i in range(N_params):
    new_V_A = V_A + step_size * i
    pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())  # You can customize the number of processes

    # Create a list of arguments for each iteration
    args_list = [
        (iter_num, V_A, theta_A, ABL_arr, ILD_arr, rate_lambda, T_0, theta_E, Z_E, t_A_aff, t_E_aff, t_motor, L, t_stim_and_led_tuple, new_V_A, N_print, dt)
        for iter_num in range(N_sim)
    ]

    # Use pool.map to distribute the work among the processes
    sim_results = pool.map(simulate_wrapper, args_list)

    pool.close()  # Close the pool to release resources
    pool.join()  # Join to make sure all processes are done

    # Print current parameters
    print(f'V_A: {V_A}, new_V_A: {new_V_A}')

    # Save the results
    V_A_change_results_dict[i] = {'results': sim_results, 'new_V_A': new_V_A, 'V_A': V_A}


os id: 55233, In iter_num: 0, ABL: 40, ILD: 16.0, t_stim: 0.538315529636573
os id: 55234, In iter_num: 1000, ABL: 20, ILD: 16.0, t_stim: 0.891075732707753
os id: 55235, In iter_num: 2000, ABL: 40, ILD: -2.0, t_stim: 1.15791543176317
os id: 55236, In iter_num: 3000, ABL: 40, ILD: -8.0, t_stim: 0.650179416871224
os id: 55238, In iter_num: 4000, ABL: 60, ILD: -2.0, t_stim: 0.585912285717598
os id: 55239, In iter_num: 5000, ABL: 60, ILD: 16.0, t_stim: 0.593105218131418
os id: 55240, In iter_num: 6000, ABL: 40, ILD: 4.0, t_stim: 0.267771721283042
os id: 55241, In iter_num: 7000, ABL: 60, ILD: 1.0, t_stim: 1.10930354506007
os id: 55243, In iter_num: 8000, ABL: 60, ILD: -4.0, t_stim: 1.25113570893279
os id: 55244, In iter_num: 9000, ABL: 40, ILD: 16.0, t_stim: 1.26847919025604
os id: 55245, In iter_num: 10000, ABL: 60, ILD: 4.0, t_stim: 0.725634423247934
os id: 55247, In iter_num: 11000, ABL: 60, ILD: 4.0, t_stim: 0.654322533362508
os id: 55246, In iter_num: 12000, ABL: 20, ILD: 1.0, t_stim: 

In [18]:
N_sim

100000